<a href="https://colab.research.google.com/github/Curro-Gonz/Airport-Signs/blob/main/trainingtfg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
from google.colab.patches import cv2_imshow

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.4/78.2 GB disk)


In [5]:
# "DRIVE" folder access from Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd {HOME}

from roboflow import Roboflow
rf = Roboflow(api_key="AE8R6X7eoyovtmBkFMSl")
project = rf.workspace("curro-p77xe").project("airport-signals")
version = project.version(4)
dataset = version.download("yolov8")


/content
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
model = YOLO("yolov8n.pt")
results = model.train(data= os.path.join("/content/Airport-signals-4", "data.yaml"), epochs = 100)

100%|██████████| 6.23M/6.23M [00:00<00:00, 103MB/s]
New https://pypi.org/project/ultralytics/8.2.45 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Airport-signals-4/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buf

In [8]:
!yolo task=detect mode=val model="/content/runs/detect/train/weights/best.pt" data="/content/Airport-signals-4/data.yaml"

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /content/Airport-signals-4/valid/labels.cache... 102 images, 6 backgrounds, 0 corrupt: 100% 102/102 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 7/7 [00:07<00:00,  1.01s/it]
                   all        102        354      0.892      0.875      0.917      0.643
              Airplane        102         64      0.879      0.798      0.888      0.564
             Direction        102         51      0.917      0.866      0.929      0.679
Enhanced taxiway centerline        102         36      0.815      0.917      0.914      0.682
              Location        102         59      0.895      0.863      0.916      0.637
        Mandatory sign        102         69       0.91       0.87      0.935      0.693
                Person        102      

In [9]:
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' conf = 0.5 source='/content/Airport-signals-4/test/images'

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs

image 1/55 /content/Airport-signals-4/test/images/172237-13037639_jpg.rf.80fdecebd67caa3dc7b041b690c0a39b.jpg: 640x640 1 Direction, 10.3ms
image 2/55 /content/Airport-signals-4/test/images/20231003_054930000_iOS_MOV-2_jpg.rf.e2d83c87752d2255d0ec888302d75db2.jpg: 640x640 5 Airplanes, 1 Enhanced taxiway centerline, 1 Runway holding position, 7.4ms
image 3/55 /content/Airport-signals-4/test/images/20231003_054930000_iOS_MOV-3_jpg.rf.b29dc509bc156987802cdbbc0c9baed4.jpg: 640x640 5 Airplanes, 1 Enhanced taxiway centerline, 1 Runway holding position, 9.1ms
image 4/55 /content/Airport-signals-4/test/images/20231003_094601000_iOS_MOV-1_jpg.rf.d09db8bd1401dff834ccc32b4fdb9d6e.jpg: 640x640 1 Location, 7.3ms
image 5/55 /content/Airport-signals-4/test/images/20231003_094601000_iOS_MOV-23_jpg.rf.382596ef819ea1e2109446c4bb7eccb9.jpg: 

In [11]:
import shutil
# Ruta del directorio que quieres copiar
source_dir = '/content/runs'
# Ruta del directorio de destino en Google Drive
dest_dir = '/content/drive/MyDrive/runs'

# Copiar el directorio
shutil.copytree(source_dir, dest_dir)

'/content/drive/MyDrive/runs'